In [4]:
import os

from agents.realtime import (
    RealtimeAgent,
    RealtimeModelConfig,
    RealtimePlaybackTracker,
    RealtimeRunner,
)
from dotenv import load_dotenv

In [5]:
if not load_dotenv():
    print("No .env file found")

api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("No OPENAI_API_KEY found in environment variables")

In [6]:
agent = RealtimeAgent(
    name="Test Agent", instructions="すべての語尾に!!!をつけて話してください。"
)

In [7]:
runner = RealtimeRunner(agent)

In [8]:
session = await runner.run(
    model_config=RealtimeModelConfig(
        api_key=api_key,
        initial_model_settings={
            "input_audio_format": "g711_ulaw",
            "output_audio_format": "g711_ulaw",
            "turn_detection": {
                "type": "semantic_vad",
                "interrupt_response": True,
                "create_response": True,
            },
        },
        playback_tracker=RealtimePlaybackTracker(),
    )
)

In [9]:
await session.enter()

In [11]:
await session.send_message("こんにちは、元気ですか？")

In [12]:
session

In [13]:
from logging import INFO, Formatter, Logger, StreamHandler, getLogger
from logging.handlers import RotatingFileHandler
from pathlib import Path


def get_logger(name: str, log_file: str) -> Logger:
    """
    log_fileに書き込む
    """
    logger = getLogger(name)

    # 既にハンドラーが設定されている場合はそのまま返す
    if logger.hasHandlers():
        return logger

    # フォーマッターの作成
    formatter = Formatter(
        "%(levelname)s %(asctime)s - %(filename)s:l%(lineno)d - %(message)s",
        datefmt="%Y%m%d %H:%M:%S",
    )

    # コンソールハンドラーの設定
    console_handler = StreamHandler()
    console_handler.setFormatter(formatter)
    console_handler.setLevel(INFO)
    logger.addHandler(console_handler)

    # ファイルハンドラーの設定
    # ログファイルのディレクトリを作成
    log_path = Path(log_file)
    log_dir = log_path.parent
    log_dir.mkdir(parents=True, exist_ok=True)

    # ローテーティングファイルハンドラーの作成
    file_handler = RotatingFileHandler(
        log_path,
        maxBytes=10 * 1024 * 1024,  # 10MB
        backupCount=5,  # 5個のバックアップ
        encoding="utf-8",
    )
    file_handler.setFormatter(formatter)
    file_handler.setLevel(INFO)
    logger.addHandler(file_handler)

    # ロガーのレベル設定
    logger.setLevel(INFO)

    return logger

In [14]:
logger = get_logger("audio_service", "logs/audio_service.log")

In [15]:
logger.info("This is an info message")

INFO 20251002 18:51:09 - 2532040493.py:l1 - This is an info message


In [16]:
logger.warning("This is a warning message")

WARNING 20251002 18:51:18 - 1885086853.py:l1 - This is a warning message
